In [131]:
import numpy as np
import torch 
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, BertForNextSentencePrediction
from transformers import BertModel
 
#MODEL_PATH = './Transformer_Bert/bert_base_chinese'
MODEL_PATH = './Transformer_Bert/FinBERT_L-12_H-768_A-12_pytorch'

# a. 通过词典导入分词器
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
# b. 导入配置文件
model_config = BertConfig.from_pretrained(MODEL_PATH)
# c. 导入Msaked
moder_mask = BertForMaskedLM.from_pretrained(MODEL_PATH)
# 修改配置
model_config.output_hidden_states = True
model_config.output_attentions = True
# 通过配置和路径导入模型
bert_model = BertModel.from_pretrained(MODEL_PATH, config = model_config)

In [ ]:
# 任务一：遮蔽语言模型
# BERT 在预训练中引入 [CLS] 和 [SEP] 标记句子的 开头和结尾
samples = ['[CLS] 中国的首都是[MASK]里？ [SEP] 北京是 [MASK] 国的[MASK]。 [SEP]'] # 准备输入模型的语句
tokenizer_text = [tokenizer.tokenize(i) for i in samples] # 将句子分割成一个个token，即一个个汉字和分隔符
#print(tokenizer_text) 

input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenizer_text]
input_ids = torch.LongTensor(input_ids)
#print(input_ids)

outputs = moder_mask(input_ids)
prediction_scores = outputs[0]                 # prediction_scores.shape=torch.Size([1, 21, 21128])
sample = prediction_scores[0].detach().numpy() # (21, 21128)  

# 21为序列长度，pred代表每个位置最大概率的字符索引
pred = np.argmax(sample, axis=1)  # (21,)
# ['，', '中', '国', '的', '首', '都', '是', '哪', '里', '？', '。', '北', '京', '是', '中', '国', '的', '首', '都', '。', '。']
print(tokenizer.convert_ids_to_tokens(pred))

In [139]:
# 定义计算相似度的函数
def calc_similarity(s1, s2):
    # 对句子进行分词，并添加特殊标记
    inputs = tokenizer([s1, s2], return_tensors='pt', padding=True, truncation=True)

    # 将输入传递给BERT模型，并获取输出
    with torch.no_grad():
        outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    # 计算余弦相似度，并返回结果
    sim = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    return sim

# 测试函数
s1 = "[CLS]复旦团队发布国内首个类ChatGPT模型MOSS[SEP]"
s2 = "[CLS]上海复旦复华科技股份有限公司所从事的主要业务包括药业、软件和园区三大产业板块。公司的主要产品为消化系统药物、神经系统用药物、抗肿瘤药物、循环系统药物、调节免疫功能药物、金融保险系统软件产品、复华园区海门园配套住宅项目（复华文苑）。公司以复旦大学的品牌、学科、科研、人才等优势为依托，具有科技成果转化生产力的有利环境；公司上市二十多年，拥有良好的品牌形象和一定的市场知名度。目前，公司主营业务结构清晰，药业、软件、园区等三大产业均为国家鼓励发展的重点产业，发展前景看好，主要产业在各自专注的细分市场能够提供富有自身特色的产品和服务，在相关领域拥有较高的知名度及一定的行业地位。[SEP]"
similarity = calc_similarity(s1, s2)
print(f"相似度：{similarity:.4f}")

相似度：0.6335
